# Hybrid Zonotopes

In [ ]:
# Generic TLT imports
from pyspect import *
from pyspect.langs.ltl import *
# Hybrid Zonotope imports
from hz_reachability.hz_impl import HZImpl
from hz_reachability.systems.cars import CarLinearModel2D, CarLinearModel4D
from hz_reachability.shapes import HZShapes
from hz_reachability.spaces import EmptySpace

TLT.select(ContinuousLTL)

## Environment definition

In [ ]:
# Option 1: Use the existing set templates or cretate your own (Not implemented for HZ yet).
# e.g., state_space = ReferredSet('state_space')

# Option 2: Use the generic Set method to import any custom shape
shapes = HZShapes()
center = Set(shapes.center())
road_west = Set(shapes.road_west())
road_east = Set(shapes.road_east())
road_north = Set(shapes.road_north())
road_south = Set(shapes.road_south())

## Definitions

### Task

In [ ]:
# Example task: Stay in road_e, or road_n UNTIL you REACH exit_n.
task = Until(Or(road_east, road_north), center)

### Dynamics

In [ ]:
reach_dynamics = CarLinearModel2D()

### Implementation

In [ ]:
# Hybrid Zonotope implementation
space = EmptySpace()
space.remove_redundant = True
impl = HZImpl(dynamics=reach_dynamics, space = space, time_horizon = 2)

## Solve

- `construct(task)`: Take an LTL, a set, or a lazy set, or an already constructed TLT and make sure it is a valid TLT object. Basically construct the compute graph for the given task.
- `realize(impl)`: initiates the actual computations.
- `out`: The final set in your specific set implementation. e.g., it would be a hybrid zonotope.

In [ ]:
# Solve the problem - Find the states that can satisfy the task
out = TLT.construct(road_east).realize(impl)
out = space.zono_op.redundant_gc_hz(out)
out = space.zono_op.redundant_c_hz(out)

# From string s, shift lines to the right by n spaces
def shift_lines(s, n):
    return '\n'.join([' '*n + l for l in s.split('\n')])

def print_hz(hz):
    dim = lambda m: "x".join(map(str, m.shape))
    print(f'Gc<{dim(hz.Gc)}>', shift_lines(str(hz.Gc), 2), sep='\n')
    print(f'Gb<{dim(hz.Gb)}>', shift_lines(str(hz.Gb), 2), sep='\n')
    print(f'C<{dim(hz.C)}>', shift_lines(str(hz.C), 2), sep='\n')
    print(f'Ac<{dim(hz.Ac)}>', shift_lines(str(hz.Ac), 2), sep='\n')
    print(f'Ab<{dim(hz.Ab)}>', shift_lines(str(hz.Ab), 2), sep='\n')
    print(f'b<{dim(hz.b)}>', shift_lines(str(hz.b), 2), sep='\n')

def save_hz(hz, directory='.'):
    from scipy.io import savemat
    from pathlib import Path
    Path(directory).mkdir(parents=True, exist_ok=True)
    savemat(f'{directory}/Gc.mat', {'Gc': hz.Gc})
    savemat(f'{directory}/Gb.mat', {'Gb': hz.Gb})
    savemat(f'{directory}/C.mat', {'C': hz.C})
    savemat(f'{directory}/Ac.mat', {'Ac': hz.Ac})
    savemat(f'{directory}/Ab.mat', {'Ab': hz.Ab})
    savemat(f'{directory}/b.mat', {'b': hz.b})

# save_hz(out, 'out-remove-redundant')
# print_hz(out)

# stuff

$$\mathcal{Z} = \{ c + G \xi \mid A \xi = b, |\xi|_\infty \leq 1 \}$$
$$ ... = \{ c + G (A^\dagger b + N_A \eta) \mid |\eta|_\infty \leq 1 \}$$
$$ ... = \{ c + G A^\dagger b + G N_A \eta \mid |\eta|_\infty \leq 1 \}$$
$$ ... = \{ x = c' + G' \eta \mid |\eta|_\infty \leq 1 \}, c' = c + G A^\dagger b, G' = G N_A$$
$$ ... = \{ x \mid | G'^\dagger (x - c') |_\infty \leq 1 \}$$


In [ ]:
import plotly.graph_objects as go

# Function to add vectors to the plot
def add_vector(fig, vector, offset, color, name):
    fig.add_trace(go.Scatter3d(
        x=[offset[0], offset[0] + vector[0]], y=[offset[1], offset[1] + vector[1]], z=[offset[2], offset[2] + vector[2]],
        mode="lines+markers",
        marker=dict(size=4),
        line=dict(width=4, color=color),
        name=name
    ))

def plot_3d_boolean_tessellation(fig, bool_grid, x_vals, y_vals, z_vals, voxel_size=1, color="blue", opacity=0.1):
    """
    Plots a 3D boolean tessellation grid using Plotly.
    
    Parameters:
        bool_grid (np.ndarray): 3D numpy array of boolean values (shape: Nx, Ny, Nz).
        x_vals (np.ndarray): 1D array of X coordinates (same length as bool_grid.shape[0]).
        y_vals (np.ndarray): 1D array of Y coordinates (same length as bool_grid.shape[1]).
        z_vals (np.ndarray): 1D array of Z coordinates (same length as bool_grid.shape[2]).
        voxel_size (float): Size of each cube (optional).
        color (str): Color of the voxels (optional).
        opacity (float): Opacity of the voxels (optional).
    """
    Nx, Ny, Nz = bool_grid.shape  # Get the grid dimensions

    # Iterate through the grid and plot only the 'True' voxels
    for i in range(Nx):
        for j in range(Ny):
            for k in range(Nz):
                if bool_grid[i, j, k]:  # If the voxel is 'True', plot it
                    x, y, z = x_vals[i], y_vals[j], z_vals[k]
                    fig.add_trace(go.Mesh3d(
                        x=[x, x+voxel_size, x+voxel_size, x, x, x+voxel_size, x+voxel_size, x],
                        y=[y, y, y+voxel_size, y+voxel_size, y, y, y+voxel_size, y+voxel_size],
                        z=[z, z, z, z, z+voxel_size, z+voxel_size, z+voxel_size, z+voxel_size],
                        i=[0, 0, 0, 1, 1, 3, 2, 4, 4, 5, 5, 6],
                        j=[1, 2, 3, 3, 5, 2, 6, 5, 6, 6, 7, 7],
                        k=[3, 3, 1, 2, 4, 6, 7, 7, 5, 4, 4, 5],
                        color=color,
                        opacity=opacity
                    ))

    # Set figure layout
    fig.update_layout(
        scene=dict(
            xaxis=dict(title="X", range=[x_vals.min(), x_vals.max()]),
            yaxis=dict(title="Y", range=[y_vals.min(), y_vals.max()]),
            zaxis=dict(title="Z", range=[z_vals.min(), z_vals.max()])
        ),
        title="3D Boolean Tessellation Grid with Custom Coordinates",
        showlegend=False
    )

    import numpy as np

def plot_3d_boolean_isosurface(fig, bool_grid, x_vals, y_vals, z_vals, color="blue", opacity=0.5):
    """
    Plots a 3D boolean isosurface representation using Plotly.
    
    Parameters:
        fig (go.Figure): The Plotly figure object to add the isosurface to.
        bool_grid (np.ndarray): 3D numpy array of boolean values (shape: Nx, Ny, Nz).
        x_vals (np.ndarray): 1D array of X coordinates (same length as bool_grid.shape[0]).
        y_vals (np.ndarray): 1D array of Y coordinates (same length as bool_grid.shape[1]).
        z_vals (np.ndarray): 1D array of Z coordinates (same length as bool_grid.shape[2]).
        color (str): Color of the isosurface (optional).
        opacity (float): Opacity of the isosurface (optional).
    """
    # Convert boolean grid to numerical values (1 for True, 0 for False)
    numerical_grid = bool_grid.astype(float)
    
    # Create the isosurface plot
    fig.add_trace(go.Isosurface(
        x=np.repeat(x_vals, len(y_vals) * len(z_vals)),
        y=np.tile(np.repeat(y_vals, len(z_vals)), len(x_vals)),
        z=np.tile(z_vals, len(x_vals) * len(y_vals)),
        value=numerical_grid.flatten(),
        isomin=0.5,  # Threshold for rendering the isosurface
        isomax=1.0,
        surface_count=1,  # Single surface rendering
        colorscale=[[0, color], [1, color]],
        opacity=opacity
    ))
    
    # Set figure layout
    fig.update_layout(
        scene=dict(
            xaxis=dict(title="X", range=[x_vals.min(), x_vals.max()]),
            yaxis=dict(title="Y", range=[y_vals.min(), y_vals.max()]),
            zaxis=dict(title="Z", range=[z_vals.min(), z_vals.max()])
        ),
        title="3D Boolean Isosurface Representation",
        showlegend=False
    )


In [ ]:
import numpy as np
import hj_reachability.shapes as shp
from numpy.linalg import matrix_rank
from scipy.linalg import null_space, svd, pinv

out = TLT(task).realize(impl)
c, G, A, b = out.C, out.Gc, out.Ac, out.b
nz = G.shape[0]


NG = null_space(G)

Gp = A @ NG

# x = c + G g,   A g = b,    |g| <= 1 

# g = G^ (x - c) + NG eta
# A G^ (x - c) + A NG eta = b  <=>  A' eta = b'
# |G^ (x - c) + NG eta| <= 1   <=>  [
#     +(G^ (x - c) + NG eta) <= 1
#     -(G^ (x - c) + NG eta) <= 1
# ]  <=>  [
#     + NG eta <= - G^ (x - c)
#     - NG eta <= + G^ (x - c)
# ]  <=>  [
#     + A NG eta <= - A G^ (x - c)
#     - A NG eta <= + A G^ (x - c)
# ]  <=>  [
#     + A' eta <= - A G^ (x - c)
#     - A' eta <= + A G^ (x - c)
# ]

# [
#     + b' <= - A G^ (x - c)
#     - b' <= + A G^ (x - c)
# ]
# where 
#   b' = A NG eta

NA = null_space(A)
NG = null_space(G)

print('NA.shape =', NA.shape, 'R(NA) =', matrix_rank(NA))
print('NG.shape =', NG.shape, 'R(NG) =', matrix_rank(NG))

print(NG)

In [ ]:
## CZ -> CZ -> Mask

import numpy as np
import hj_reachability.shapes as shp
from scipy.linalg import null_space, svd, pinv

c = np.array([[2, 2, 2]]).T
G = np.array([
    [1, 2, 0,  1],
    [0, 1, 2, -1],
    [1, 0, 2,  1],
])
A = np.array([
    [1, 0, -1, 0],
])
b = np.array([[0]]).T

out = TLT(road_east).realize(impl)
c, G, A, b = out.C, out.Gc, out.Ac, out.b

## (inactive) Reorder G

# P = np.identity(3)

# PG = P @ G
# m = np.linalg.norm(PG, axis=0)
# ix = m[::-1].argsort()

# print(f'{m = }')
# print(f'{ix = }')

## Step 1: Compute null space of A

N_A = null_space(A)

print(f'{N_A.shape = }')

## Step 2: Compute the effective generator matrix

GN = G @ N_A

print(f'{GN.shape = }')

## Random idea:

coords = [
    np.linspace(-1, 5, 21),
    np.linspace(-1, 5, 21),
    np.linspace(-1, 5, 21),
    np.linspace(-1, 5, 21),
    np.linspace(-1, 5, 21),
]

X = np.array(np.meshgrid(*coords))

print(f'{c.shape = }')
print(f'{G.shape = }')
print(f'{A.shape = }')
print(f'{pinv(A).shape = }')
print(f'{b.shape = }')

cp = c + G @ pinv(A) @ b
Gp = G @ N_A

print(f'{X.shape = }')
print(f'{cp.shape = }')
print(f'{Gp.shape = }')
print(f'{pinv(Gp).shape = }')

N_G = null_space(G)

print(f'{N_G.shape = }')

sol = shp.tmul(pinv(Gp), X - cp.reshape(-1, *[1] * len(coords)))

print(f'{sol.shape = }')
print(f'{sol.min() = }', ';', f'{sol.max() = }')

## Plot Col. vectors

mask = np.abs(sol).sum(axis=0) <= 1

fig = go.Figure(layout=dict(scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
                            title="Subspaces of A",
                            showlegend=True,
                            width=800, height=800))

plot_3d_boolean_isosurface(fig, mask, *coords)

# add_vector(fig, G[:, 0], c, 'red', 'Col. 1')
# add_vector(fig, G[:, 1], c, 'green', 'Col. 2')
# add_vector(fig, G[:, 2], c, 'blue', 'Col. 3')
# add_vector(fig, G[:, 3], c, 'yellow', 'Col. 4')

fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.linalg import svd, null_space

# Define a 3x3 matrix (change this to experiment with different transformations)
# A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Compute Column Space (from SVD)
U, S, Vt = svd(A)
rank_A = np.sum(S > 1e-10)
col_space = U[:, :rank_A]  # First 'rank' columns of U

# Compute Row Space (rows of A)
row_space = Vt[:rank_A, :].T  # First 'rank' rows of Vt

# Compute Null Space (Solve Ax = 0)
null_A = null_space(A)  # Null space of A

# Compute Left Null Space (Solve A^T y = 0)
null_At = null_space(A.T)  # Null space of A^T

# Initialize a Plotly figure
fig = go.Figure()

# Function to add vectors to the plot
def add_vector(fig, vector, color, name):
    fig.add_trace(go.Scatter3d(
        x=[0, vector[0]], y=[0, vector[1]], z=[0, vector[2]],
        mode="lines+markers",
        marker=dict(size=4),
        line=dict(width=4, color=color),
        name=name
    ))

# Plot Column Space (Blue)
for i in range(col_space.shape[1]):
    add_vector(fig, col_space[:, i], "blue", "Column Space")

# Plot Row Space (Green)
for i in range(row_space.shape[1]):
    add_vector(fig, row_space[:, i], "green", "Row Space")

# Plot Null Space (Red)
if null_A.shape[1] > 0:
    for i in range(null_A.shape[1]):
        add_vector(fig, null_A[:, i], "red", "Null Space")

# Plot Left Null Space (Purple)
if null_At.shape[1] > 0:
    for i in range(null_At.shape[1]):
        add_vector(fig, null_At[:, i], "purple", "Left Null Space")

# Set axis labels
fig.update_layout(
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z",
    ),
    title="Subspaces of A",
    showlegend=True,
    width=800, height=800,
)

# Show interactive plot
fig.show()


## Conversion

### HJ Setup

In [ ]:
import hj_reachability as hj
import hj_reachability.shapes as shp

from pyspect.impls.hj_reachability import TVHJImpl
from hj_reachability.systems import Bicycle4D
from pyspect.plotting.levelsets import *

from math import pi

# Define origin and size of area, makes it easier to scale up/down later on 
X0, XN = -1.2, 2.4
Y0, YN = -1.2, 2.4
Z0, ZN = -1.2, 2.4

min_bounds = np.array([   X0,    Y0])
max_bounds = np.array([XN+X0, YN+Y0])
grid_space = (51, 51)

# min_bounds = np.array([   X0,    Y0,    Z0])
# max_bounds = np.array([XN+X0, YN+Y0, ZN+Z0])
# grid_space = (51,51,51)

# min_bounds = np.array([   X0,    Y0, -pi, 1.0])
# max_bounds = np.array([XN+X0, YN+Y0, +pi, 0.0])
# grid_space = (31, 31, 21, 11)

grid = hj.Grid.from_lattice_parameters_and_boundary_conditions(hj.sets.Box(min_bounds, max_bounds),
                                                               grid_space)

dynamics = dict(cls=None)

hj_impl = TVHJImpl(dynamics, grid, 3)
hj_impl.set_axes_names('t', 'x', 'y')

### Method

In [ ]:
import jax
import jax.numpy as jnp

@jax.jit
def infmax_conv(VA, VB):

    # We'll flatten all (i,j) into an array of shape [Nx*Ny, 2].
    IJ_x = jnp.meshgrid(*[jnp.arange(n) for n in VA.shape],
                      indexing='ij')
    indices = jnp.stack(IJ_x, axis=-1).reshape(-1, 2)

    def infmax_conv_cell(carry, ij_x):
        """
        Compute VC(x) = min_y max{ VA(y), VB(x-y) }
        for a particular (ij..).
        """
        def body(k, current_min):
            # k goes from 0 to Nx*Ny
            # Decompose k into (a, b)
            a = k // Ny
            b = k % Ny
            
            i2 = i - a
            j2 = j - b
            
            # Check out-of-bounds for (i2, j2)
            valid = (0 <= i2) & (i2 < Nx) & (0 <= j2) & (j2 < Ny)
            val = jnp.where(
                valid,
                jnp.maximum(VA[a, b], VB[i2, j2]),
                jnp.inf
            )
            return jnp.minimum(current_min, val)
        
        ## Loop over y

        return jax.lax.fori_loop(
            lower=0,
            upper=Nx*Ny,
            body_fun=body_y,
            init_val=jnp.inf
        )

    ## Loop over x
    return jax.lax.scan(infmax_conv_cell, None, indices)[1].reshape(VA.shape)

In [ ]:
from scipy.ndimage import convolve
import numba as nb

def crop_mask(mask):
    """Reduce a binary mask to the smallest bounding box that includes all True values.

    Args:
        mask (ndarray): N-D boolean mask.

    Returns:
        cropped_mask (ndarray): Cropped version of the mask.
        slices (tuple): Tuple of slices that define the cropped region.
    """
    if not np.any(mask):  # Check if all False
        return mask, tuple(slice(0, 0) for _ in range(mask.ndim))  # Empty region
    
    # Find min/max indices for each axis
    slices = tuple(
        slice(np.min(indices), np.max(indices) + 1)
        for indices in (np.where(mask) if mask.ndim > 1 else (np.where(mask)[0],))
    )
    
    return mask[slices], slices

def generator(hz, i):
    Gc, Gb, C, Ac, Ab, b = hz
    nc = Ac.shape[0]
    ng = Ac.shape[1]

    C = C.reshape(-1)
    b = b.reshape(-1)
    
    # --- Continuous Constraints ---

    data = -np.inf * np.ones(grid.shape)
    for i in range(nc):
        data = np.max(
            data,
            shp.hyperplane(grid, normal=Ac[i], offset=[0]*ng, const=b[i]),
        )

    # --- Continuous Generators ---

    g = Gc[:, i:i+1]
    data = np.max(
        data,
        shp.intersection(
            shp.cylinder(grid, r=GEN_WIDTH, c=C, axis=g),
            shp.hyperplane(grid, normal=+g, offset=C+g),
            shp.hyperplane(grid, normal=-g, offset=C-g),
        ),
    )

def hz2hj(hz):

    Gc, Gb, C, Ac, Ab, b = hz
    ng = Gc.shape[1]

    ## Generators

    I = (generator(hz, 0) <= 0).astype(int)

    for i in np.arange(1, ng):
        K = (generator(hz, i) <= 0).astype(int)
        K = crop_mask(K)[0]

        I = convolve(I, K, mode='constant', cval=0) > 0

    vf = 0.5 - (I > 0)
    return vf


// Disabled

z = center(None)
z = out

Gc, Gb, C, Ac, Ab, b = z.astuple()
ng, nb, nc = z.ng, z.nb, z.nc

nx = (21, 2) # grid_space
beta, delta = np.meshgrid(np.linspace(-1, 1, nx[0]), np.linspace(0, 1, nx[1]))

gen_c = np.stack([beta for _ in range(ng)], axis=0) if ng > 0 else np.array([]).reshape(0, *beta.shape)
gen_b = np.stack([delta for _ in range(nb)], axis=0) if nb > 0 else np.array([]).reshape(0, *delta.shape)
gen = np.vstack([gen_c, gen_b])

print()
print('beta<?>:', beta.shape)
print('delta<?>:', delta.shape)
print('gen_c<ng, ?>:', gen_c.shape)
print('gen_b<nb, ?>:', gen_b.shape)
print('gen<ng+nb, ?>:', gen.shape)

b = b.reshape(-1, 1, 1) # (nc, 1, 1) for broadcasting over grid
A = np.hstack([Ac, Ab]) # (nc, ng+nb)

print()
print('b<nc, ?>:', b.shape)
print('Ac<nc, ng>:', Ac.shape)
print('Ab<nc, nb>:', Ab.shape)
print('A<nc, ng+nb>:', A.shape)

# Loop over constraints and iteratively apply them
mask = np.ones((ng+nb, *nx), dtype=bool)
for i in range(nc):
    mask &= np.abs( shp.tmul(A[i], gen) - b[i] ) <= 0.1

# gen_c = gen_c[mask]
# gen_b = gen_b[mask]

# print(constr.min(), constr.max())
# # plot histogram of constr
# fig, ax = plt.subplots()
# ax.hist(np.abs(constr).flatten(), bins=100)
# plt.xlim(-0.2, 0.2)
# plt.show()

print()
print('mask<ng+nb, ?>:', mask.shape)
# print('*gen_c<ng, ?>:', gen_c.shape)
# print('*gen_b<nb, ?>:', gen_b.shape)

print()
print('C<n, 1>:', C.shape)
print('Gc<n, ng>:', Gc.shape)
print('Gb<n, nb>:', Gb.shape)

x = (
    + C.reshape(-1, 1, 1)
    + shp.tmul(Gc, gen_c)
    + shp.tmul(Gb, gen_b)
)

print()
print('x:', x.shape)

#plot points in x
fig, ax = plt.subplots()
ax.scatter(x, x)
plt.show()


In [ ]:
# from hz_reachability.visualizer import ZonoVisualizer
# from hz_reachability.auxiliary_operations import ZonoOperations

# op = ZonoOperations()
# viz = ZonoVisualizer(op)

# out = op.redundant_c_hz(out)
# viz.vis_hz([out])

In [ ]:
from scipy.signal import correlate
from scipy import ndimage as ndi
from tqdm import trange

GEN_WIDTH = np.linalg.norm(grid.spacings)

## ##

fig_select = 2
fig_kwds = dict(fig_theme='Light')

sq = shp.rectangle(grid, [-.5, -.5], [+.5, +.5])

plot2D_bitmap(
    # M,
    **fig_kwds, 
    fig_enabled=fig_select==0,
) or \
plot3D_valuefun(
    # vf,
    min_bounds=min_bounds,
    max_bounds=max_bounds,
    **fig_kwds,
    fig_enabled=fig_select==1,
) or \
plot_levelsets(
    # shp.project_onto(vf, 1, 2),
    ndi.geometric_transform(sq, lambda coord: (coord[0] - 1,coord[1] + 1), cval=1.0),
    

    # (hz2hj(shapes.road_west()), dict(colorscale='blues')),
    # (hz2hj(shapes.road_east()), dict(colorscale='blues')),
    # (hz2hj(shapes.road_south()), dict(colorscale='blues')),
    # (hz2hj(shapes.road_north()), dict(colorscale='blues')),
    # (hz2hj(shapes.center().astuple()), dict(colorscale='greens')),

    # (hz2hj(out.astuple()), dict(colorscale='greens')),

    # axes = (0, 1, 2),
    min_bounds=min_bounds,
    max_bounds=max_bounds,
    # plot_func=plot3D_levelset,
    **fig_kwds,
    fig_enabled=fig_select==2,
    fig_width=500, fig_height=500,
)

In [ ]:
def foo():
    eps = 0.1
    Nx = 1
    Nd = 30

    # (Nd, Nx, ..., Nx)
    x = np.array(np.meshgrid(*[np.linspace(-1, 1, Nx)] * Nd))

    # 
    y = np.tensordot(a, x, ([0], [0])) - b
    
    mask = y <= eps

    for i, xi in enumerate(x):
        y[mask] 


    for i, k, m in zip(axes, normal, offset):
        data += k*x(i) - k*m
    return data